In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta,date
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest

# apply to airflow composer

In [2]:
#def summarize_buy_sell_trans:

In [3]:
table="`pongthorn.FinDW.bn_hist_trade`"
table_report="pongthorn.FinDW.bn_bs_summary"

dispCols=['datetimeUTC','symbol_pair','type','price','qty','total']

In [4]:
date_str_from='2022-09-01'
created_date=datetime.now()


In [5]:
 # get yesterday transaction
date_str_to=created_date.strftime('%Y-%m-%d')


# Find Summary Buy & Sell

In [6]:
bq_client = bigquery.Client()  

In [7]:
def summarize_bs(from_date,to_date):  # no return
    

    sql=f"""select datetimeUTC,symbol_pair,symbol,pair,type,price,qty,total from {table} 
        where datetimeUTC >='{from_date}' and  datetimeUTC<'{to_date}' 
        order by datetimeUTC desc """
    # print(sql)

    query_job =bq_client.query(query=sql)
    df_trasns= query_job.to_dataframe() 
    
    #print(df_trasns[dispCols])
    if df_trasns.empty==False:
        print(f"List trade transaction between {from_date} and {to_date}")
    
        print("Agg transaction")
        groupby_cols=['symbol','type']
        agg_cols=['qty','total']
        df_summary=df_trasns.groupby(groupby_cols,as_index=False)[agg_cols].sum()
        df_summary['avg_price']=df_summary['total']/df_summary['qty']
        df_summary=df_summary.round(5)
        
        
        report_date=datetime.strptime(to_date,'%Y-%m-%d')-timedelta(days=1)
        df_summary['report_date']=report_date
        
        print(df_summary)
        df_summary['created_date']=created_date

        try:
            job_config = bigquery.LoadJobConfig( write_disposition="WRITE_APPEND",)
            job = bq_client.load_table_from_dataframe(
                df_summary,table_report, job_config=job_config
            ) 
            job.result() 
            print("Total ", len(df_summary), " Add transaction to google bigquery successfully")

        except BadRequest as e:
            print("Bigquery Error\n")
            for e in job.errors:
                print('ERROR: {}'.format(e['message']))
    else:
        print(f"no trade transaction between {date_str_from} and {date_str_to}")
    
    print("=========================================================================================")
        
    return 'ok'    

In [9]:
# for migrate data
indexHistReport=pd.date_range(start='2022-09-02', end=created_date.strftime('%Y-%m-%d'),freq='d').tolist()
listHistReport=[ x.date().strftime('%Y-%m-%d')  for x in indexHistReport  ]
for hist_to in listHistReport:
  result=summarize_bs(date_str_from,hist_to)  
# for daily
# result=summarize_bs(date_str_from,date_str_to)


List trade transaction between 2022-09-01 and 2022-10-21
Agg transaction
  symbol  type       qty      total    avg_price report_date
0    BNB   BUY    0.5660  155.68050    275.05389  2022-10-20
1    BNB  SELL    0.3150   85.69750    272.05556  2022-10-20
2    BTC   BUY    0.0021   40.56791  19318.05238  2022-10-20
3    ETH   BUY    0.1928  261.55193   1356.59714  2022-10-20
4    ETH  SELL    0.1339  178.49234   1333.02721  2022-10-20
5  MATIC   BUY  293.6000  233.54488      0.79545  2022-10-20
6  MATIC  SELL  213.6000  169.27268      0.79248  2022-10-20
Total  7  Add transaction to google bigquery successfully


In [ ]:
# if __name__ == "__main__":
#  result=summarize_buy_sell_trans()
#  print(result)